In [1]:
%%prun
from vpython import *
from random import random
import numpy as np

# A model of an ideal gas with hard-sphere collisions
# Program uses numpy arrays for high speed computations

# Bruce Sherwood

win=500

Natoms = 100  # change this to have more or fewer atoms

# Typical values
L = 1 # container is a cube L on a side
gray = color.gray(0.7) # color of edges of container
mass = 4E-3/6E23 # helium mass
Ratom = 0.03 # wildly exaggerated size of helium atom
k = 1.4E-23 # Boltzmann constant
T = 300 # around room temperature
dt = 1E-5

scene = canvas(title="Gas", width=win, height=win, center=vec(L/2,L/2,L/2))
scene.range = L

deltav = 100 # binning for v histogram
vdist = graph(xmax=3000, ymax=Natoms*deltav/1000, width=win, height=0.4*win, xtitle='v', ytitle='dN')
theory = gcurve(color=color.cyan)

xaxis = curve(pos=[vec(0,0,0), vec(L,0,0)], color=gray)
yaxis = curve(pos=[vec(0,0,0), vec(0,L,0)], color=gray)
zaxis = curve(pos=[vec(0,0,0), vec(0,0,L)], color=gray)
xaxis2 = curve(pos=[vec(L,L,L), vec(0,L,L), vec(0,0,L), vec(L,0,L)], color=gray)
yaxis2 = curve(pos=[vec(L,L,L), vec(L,0,L), vec(L,0,0), vec(L,L,0)], color=gray)
zaxis2 = curve(pos=[vec(L,L,L), vec(L,L,0), vec(0,L,0), vec(0,L,L)], color=gray)

dv = 10
for v in arange(0,3001+dv,dv): # theoretical prediction
    theory.plot(v,
        (deltav/dv)*Natoms*4*pi*((mass/(2.*pi*k*T))**1.5)
                     *exp((-0.5*mass*v**2)/(k*T))*v**2*dv)

accum = []
for i in range(int(3000/deltav)): accum.append([deltav*(i+.5),0])
vdist = gvbars(color=color.red, delta=deltav )

nhisto = 0 # number of histogram snapshots to average

def barx(v):
    return int(v/deltav) # index into bars array

def interchange(v1, v2):  # remove from v1 bar, add to v2 bar
    barx1 = barx(v1)
    barx2 = barx(v2)
    if barx1 == barx2: return
    if barx1 > len(histo) or barx2 > len(histo):
        print(v1, v2, barx2, barx2)
    histo[barx1] -= 1
    histo[barx2] += 1
    
Atoms = []
poslist = []
plist = []
mlist = []
rlist = []
pavg = sqrt(2*mass*1.5*k*T) # average kinetic energy p**2/(2mass) = (3/2)kT

for i in range(Natoms):
    Lmin = 1.1*Ratom
    Lmax = L-Lmin
    x = Lmin+(Lmax-Lmin)*random()
    y = Lmin+(Lmax-Lmin)*random()
    z = Lmin+(Lmax-Lmin)*random()
    if i == 0:
        Atoms.append(sphere(pos=vec(x,y,z), radius=Ratom, color=color.yellow,
                    make_trail=True, retain=100))
    else:
        Atoms.append(sphere(pos=vec(x,y,z), radius=Ratom, color=color.cyan))
    theta = pi*random()
    phi = 2*pi*random()
    px = pavg*sin(theta)*cos(phi)
    py = pavg*sin(theta)*sin(phi)
    pz = pavg*cos(theta)
    poslist.append((x,y,z))
    plist.append((px,py,pz))
    mlist.append(mass)

pos = np.array(poslist)
p = np.array(plist)

nhisto = int(4500/deltav)
histo = []
for i in range(nhisto): histo.append(0)
histo[barx(pavg/mass)] = Natoms
count = 0
timer = clock()

def checkCollisions():
    global pos
    r = pos-pos[:,np.newaxis] # all pairs of atom-to-atom vectors
    rmag = np.sqrt(np.sum(np.square(r),-1)) # atom-to-atom scalar distances
    hit = np.less_equal(rmag,2*Ratom)-np.identity(Natoms)
    return np.sort(np.nonzero(hit.flat)[0]).tolist() # i,j encoded as i*Natoms+j

while True:
    rate(1000000)
    # Accumulate and average histogram snapshots
    for i in range(len(accum)): accum[i][1] = (nhisto*accum[i][1] + histo[i])/(nhisto+1)
##    if nhisto % 10 == 0:
##        vdist.data = accum
    nhisto += 1

    # Update all positions
    pos = pos+(p/mass)*dt

    hitlist = checkCollisions()
    
    # If any collisions took place:
    for ij in hitlist:
        i, j = divmod(ij,Natoms) # decode atom pair
        hitlist.remove(j*Natoms+i) # remove symmetric j,i pair from list
        ptot = p[i]+p[j]
        posi = pos[i]
        posj = pos[j]
        vi = p[i]/mass
        vj = p[j]/mass
        vrel = vj-vi
        a = sum(vrel*vrel)
        if a == 0: continue # exactly same velocities
        rrel = posi-posj
        b = 2*dot(rrel,vrel)
        c = sum(rrel*rrel)-4*Ratom*Ratom
        d = b**2-4*a*c
        if d < 0: continue # something wrong; ignore this rare case
        deltat = (-b+sqrt(d))/(2*a) # t-deltat is when they made contact
        posi = posi-vi*deltat # back up to contact configuration
        posj = posj-vj*deltat
        mtot = 2*mass
        pcmi = p[i]-ptot*mass/mtot # transform momenta to cm frame
        pcmj = p[j]-ptot*mass/mtot
        rrel = rrel/(sum(rrel*rrel)**0.5)
        pcmi = pcmi-2*pcmi.dot(rrel)*rrel # bounce in cm frame
        pcmj = pcmj-2*pcmj.dot(rrel)*rrel
        p[i] = pcmi+ptot*mass/mtot # transform momenta back to lab frame
        p[j] = pcmj+ptot*mass/mtot
        pos[i] = posi+(p[i]/mass)*deltat # move forward deltat in time
        pos[j] = posj+(p[j]/mass)*deltat
        interchange(sum(vi*vi)**0.5, sum(p[i]*p[i])**0.5/mass)
        interchange(sum(vj*vj)**0.5, sum(p[j]*p[j])**0.5/mass)
 
    # Bounce off walls
    outside = np.less_equal(pos,Ratom) # walls closest to origin
    p1 = p*outside
    p = p-p1+abs(p1) # force p component inward
    outside = np.greater_equal(pos,L-Ratom) # walls farther from origin
    p1 = p*outside
    p = p-p1-abs(p1) # force p component inward

    # Update positions of display objects
    for i in range(Natoms):
        tpos = pos[i]
        Atoms[i].pos = vec(tpos[0], tpos[1], tpos[2])
    
    count += 1
    if count >= 1000: break
timer = clock()-timer
print(1000*timer/count, 'ms per loop')
vdist.data = accum



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

10.985853410061925 ms per loop
 